In [1]:
import re
import requests
from bs4 import BeautifulSoup
import datetime as dt

r = requests.get('http://bensweather.com')
soup = BeautifulSoup(r.content, 'html.parser')
forecast = soup.find('table', {"background":"images/7daybg.png"})

In [2]:
forecast.find_all('img')

[<img alt="" height="18" src="wxicons/atue.png" width="110"/>,
 <img alt="" height="75" src="wxicons/showers.png" width="57"/>,
 <img alt="" height="18" src="wxicons/awed.png" width="110"/>,
 <img alt="" height="75" src="wxicons/cloudspartly.png" width="75"/>,
 <img alt="" height="18" src="wxicons/athu.png" width="110"/>,
 <img alt="" height="75" src="wxicons/sunny.png" width="75"/>,
 <img alt="" height="18" src="wxicons/afri.png" width="110"/>,
 <img alt="" height="75" src="wxicons/sunny.png" width="75"/>,
 <img alt="" height="18" src="wxicons/asat.png" width="110"/>,
 <img alt="" height="75" src="wxicons/sunny.png" width="75"/>,
 <img alt="" height="18" src="wxicons/asun.png" width="110"/>,
 <img alt="" height="75" src="wxicons/sunny.png" width="75"/>,
 <img alt="" height="18" src="wxicons/amon.png" width="110"/>,
 <img alt="" height="75" src="wxicons/cloudshigh.png" width="75"/>]

In [7]:
import re
import requests
from bs4 import BeautifulSoup
import datetime as dt
forecasts = ['sunny', 'cloudshigh','cloudspartly', 'smoke','snowshowers']

class BensWeatherForecast:
    def build(self):
        r = requests.get('http://bensweather.com')
        soup = BeautifulSoup(r.content, 'html.parser')
        forecast = soup.find('table', {"background":"images/7daybg.png"})
        my_forecast = self._initialize_forecast(forecast)
        my_forecast = self._decorate_with_weather(my_forecast, forecast)
        my_forecast = self._decorate_with_highs_and_lows(my_forecast, forecast)
        return my_forecast
        
    def _initialize_forecast(self, forecast):
        my_forecast = []

        counter=0
        for img in forecast.find_all('img',{'width':'110'}):
            if img['src'].startswith('wxicons/a'):
                day = img['src'][9:12]
                the_date = dt.date.today() + dt.timedelta(days=counter)
                my_forecast.append(dict(day=day, dt=the_date))
                counter +=1
        return my_forecast

    def _decorate_with_weather(self, arr, forecast):

        counter = 0
#         for img in forecast.find_all('img', {'width':'75'}):
        for img in forecast.find_all('img'):
            if not img['src'].startswith('wxicons/a'):
#                 print(img['src'])
                weather = img['src'][8:].rstrip('.png')
                arr[counter]['forecast'] = weather
                counter +=1
        return arr

    def _decorate_with_highs_and_lows(self, arr, forecast):
        counter = 0
        forecast_row = forecast.find('tr', {'class':'style42'})
        for td in forecast_row.find_all('td'):
            parts = td.text.replace('\n',' ').strip().rstrip("'s").replace('°','').split('/')
            result = re.search('(\d+)', parts[0])
            if result is None: continue
            high = result[0]
            arr[counter]['high'] = int(high)
            label = parts[0].split(high)[0].strip().lower()
            arr[counter]['description'] = label
            low = parts[1].strip()
            arr[counter]['low'] = int(low)
            counter +=1

        return arr

In [8]:
my_forecast = BensWeatherForecast().build()
my_forecast

[{'day': 'tue',
  'dt': datetime.date(2022, 1, 18),
  'forecast': 'showers',
  'high': 48,
  'description': 'partly cloudy',
  'low': 20},
 {'day': 'wed',
  'dt': datetime.date(2022, 1, 19),
  'forecast': 'cloudspartly',
  'high': 50,
  'description': 'sunny',
  'low': 20},
 {'day': 'thu',
  'dt': datetime.date(2022, 1, 20),
  'forecast': 'sunny',
  'high': 48,
  'description': 'sunny',
  'low': 20},
 {'day': 'fri',
  'dt': datetime.date(2022, 1, 21),
  'forecast': 'sunny',
  'high': 45,
  'description': 'sunny',
  'low': 20},
 {'day': 'sat',
  'dt': datetime.date(2022, 1, 22),
  'forecast': 'sunny',
  'high': 49,
  'description': 'sunny',
  'low': 20},
 {'day': 'sun',
  'dt': datetime.date(2022, 1, 23),
  'forecast': 'sunny',
  'high': 51,
  'description': 'mostly sunny',
  'low': 20},
 {'day': 'mon', 'dt': datetime.date(2022, 1, 24), 'forecast': 'cloudshigh'}]